In [1]:
import pymc as pm
import numpy as np
import arviz as az
import pandas as pd

%load_ext lab_black

# Predicting Using Censored Data*

Adapted from [Unit 10: katla.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit10/katla.odc).

Data can be found [here](https://raw.githubusercontent.com/areding/6420-pymc/main/data/r.txt).

In 2010 Icelandic volcano Eyjafjallajökull erupted. Nearby volcano Katla erupts more frequently.

This model attempts to predict the next Katla erruption (BUGS Book p254).

notes:

Having problems with Weibull and pm.Censored again! What is the deal here? Imputed censoring works fine.

In [2]:
# fmt: off
D = np.array(
    (1177, 1262, 1311, 1357, 1416, 1440, 1450, 1500, 
     1550, 1580, 1612, 1625, 1660, 1721, 1755, 1823, 
     1860, 1918, 1000000)
)
# fmt: on

# probabilities
ps = [1, 5, 10, 50]

# time between eruptions
t = np.diff(D)

In [6]:
with pm.Model() as m:
    α = pm.Exponential("α", 0.05)  # v in BUGS model

    σ = pm.Gamma("σ", 0.1, 0.1)
    λ = 1 / σ**α
    β = λ ** (-1 / α)

    _t = pm.Weibull.dist(α, β)
    pm.Censored("likelihood", _t, lower=None, upper=100, observed=t)

    median = pm.Deterministic("median tte", σ * np.log(2) ** (1 / α))

    for p in ps:
        pm.Deterministic(
            f"p_erupt_{p}", 1 - pm.math.exp((100 / σ) ** α - ((100 + p) / σ) ** α)
        )

    trace = pm.sample(3000, init="jitter+adapt_diag_grad")
                      
                      

Auto-assigning NUTS sampler...
INFO:pymc:Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag_grad...
INFO:pymc:Initializing NUTS using jitter+adapt_diag_grad...


SamplingError: Initial evaluation of model at starting point failed!
Starting values:
{'α_log__': array(2.90213014), 'σ_log__': array(0.43606281)}

Initial evaluation results:
{'α': -1.0, 'σ': -2.59, 'likelihood': -inf}

This worked fine in PyMC 4, but since the update to 5 it doesn't work. Imputed censoring method (below) works fine for now, but will need to update it to the new, non-deprecated method at some point.

In [ ]:
az.summary(trace)

In [7]:
t_uncens = t[:-1]

In [8]:
with pm.Model() as m:
    # α = pm.Uniform("α", 0, 10) # getting divide by 0 errors
    α = pm.TruncatedNormal("α", mu=0, sigma=5, lower=0)  # v in BUGS model

    σ = pm.Gamma("σ", 0.001, 0.001)
    λ = (1 / σ) ** α
    β = λ ** (-1 / α)

    impute_censored = pm.Bound("impute_censored", pm.Weibull.dist(α, β), lower=100)

    pm.Weibull("uncensored", α, β, observed=t_uncens)

    median = pm.Deterministic("median tte", σ * np.log(2) ** (1 / α))

    for p in ps:
        pm.Deterministic(
            f"p_erupt_{p}", 1 - pm.math.exp((100 / σ) ** α - ((100 + p) / σ) ** α)
        )

    trace = pm.sample(3000)

/Users/aaron/mambaforge/envs/pymc_env2/lib/python3.11/site-packages/pymc/distributions/bound.py:186: FutureWarning: Bound has been deprecated in favor of Truncated, and will be removed in a future release. If Truncated is not an option, Bound can be implemented byadding an IntervalTransform between lower and upper to a continuous variable. A Potential that returns negative infinity for values outside of the bounds can be used for discrete variables.
  warnings.warn(
Auto-assigning NUTS sampler...
INFO:pymc:Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
INFO:pymc:Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
INFO:pymc:Multiprocess sampling (4 chains in 4 jobs)
NUTS: [α, σ, impute_censored]
INFO:pymc:NUTS: [α, σ, impute_censored]


Sampling 4 chains for 1_000 tune and 3_000 draw iterations (4_000 + 12_000 draws total) took 2 seconds.
INFO:pymc:Sampling 4 chains for 1_000 tune and 3_000 draw iterations (4_000 + 12_000 draws total) took 2 seconds.
There were 11 divergences after tuning. Increase `target_accept` or reparameterize.
ERROR:pymc:There were 11 divergences after tuning. Increase `target_accept` or reparameterize.


In [9]:
az.summary(trace)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
α,2.011,0.389,1.286,2.737,0.005,0.004,6034.0,6738.0,1.0
σ,54.694,7.285,41.135,68.236,0.089,0.063,6704.0,6643.0,1.0
impute_censored,114.822,16.502,100.003,142.952,0.198,0.140,5526.0,4500.0,1.0
median tte,45.328,6.484,32.834,57.256,0.080,0.056,6542.0,6311.0,1.0
p_erupt_1,0.071,0.031,0.023,0.130,0.000,0.000,6699.0,6570.0,1.0
p_erupt_5,0.306,0.112,0.110,0.514,0.001,0.001,6681.0,6558.0,1.0
p_erupt_10,0.514,0.151,0.228,0.790,0.002,0.001,6659.0,6546.0,1.0
p_erupt_50,0.959,0.064,0.843,1.000,0.001,0.001,6525.0,6537.0,1.0


In [10]:
%load_ext watermark
%watermark -n -u -v -iv -p pytensor

Last updated: Wed Mar 22 2023

Python implementation: CPython
Python version       : 3.11.0
IPython version      : 8.9.0

pytensor: 2.10.1

pymc  : 5.1.2
arviz : 0.15.1
numpy : 1.24.2
pandas: 1.5.3

